In [1]:

# check linked creators, for now that no two Wikidata entities should share ACMI creator link.

import hashlib
import json
import numpy
import pandas
import pathlib
import pydash
import requests
import time
import tqdm

def value_extract(row, column):

    ''' Extract dictionary values. '''
    
    return pydash.get(row[column], 'value')

def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    response = requests.get(service, params={'format': 'json', 'query': query}, timeout=120)
    results = pydash.get(response.json(), 'results.bindings')
    data_frame = pandas.DataFrame.from_dict(results)
    for column in data_frame.columns:
        data_frame[column] = data_frame.apply(value_extract, column=column, axis=1)
    
    return data_frame

query = '''
    select ?acmi ?wikidata
    where {
        ?wikidata wdt:P7003 ?acmi .
        filter(regex(str(?acmi), "creator")) .
        } '''

dataframe = sparql_query(query, 'https://query.wikidata.org/sparql').drop_duplicates()

mask = dataframe.drop_duplicates(subset='acmi',keep=False)
dataframe = dataframe.loc[~dataframe.acmi.isin(list(mask.acmi))]
dataframe = dataframe.sort_values(by='acmi')

print(len(dataframe))
dataframe.head()

4


,wikidata,acmi
2958,http://www.wikidata.org/entity/Q636003,creators/14128
2959,http://www.wikidata.org/entity/Q3197378,creators/14128
3273,http://www.wikidata.org/entity/Q3246627,creators/83206
3274,http://www.wikidata.org/entity/Q120308518,creators/83206
